In [440]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import pandas as pd

数据处理

In [441]:
iris = load_iris()
x, y = iris.data, iris.target
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

SVM

In [442]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf']
}

# 网格搜索
grid = GridSearchCV(SVC(), param_grid, refit=True, cv=5)
grid.fit(x_train, y_train)

# 评估模型
svm_acc = grid.score(x_test, y_test)
print(f"SVM准确率: {svm_acc:.4f}")


SVM准确率: 1.0000


逻辑回归

In [443]:
log=LogisticRegression( penalty='l2',  # L2正则化
    C=1.0,         # 正则化强度的倒数，C越小正则化越强
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=1000)
log.fit(x_train,y_train)
log_acc=log.score(x_test,y_test)#正确预测的样本数占总样本数的比例,分类任务
print(f"逻辑回归准确率: {log_acc:.4f}")

逻辑回归准确率: 0.9667


/home/liuyongze/vs/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


knn

In [444]:
knn=KNeighborsClassifier(n_neighbors=9)
knn.fit(x_train,y_train)
knn_acc=knn.score(x_test,y_test)
print(f"knn准确率: {knn_acc:.4f}")

knn准确率: 0.9667


投票集成

In [445]:


# 创建投票集成模型
voting_clf = VotingClassifier(
    estimators=[
        ('svm_clf' , SVC(kernel='linear', C=1.0, random_state=42)),
        ('knn', KNeighborsClassifier(n_neighbors=5)),
        ('lig_reg2', LogisticRegression(penalty='l2', max_iter=1000))
    ],
    voting='hard'  # 硬投票
)
voting_clf.fit(x_train, y_train)
voting_acc = voting_clf.score(x_test, y_test)
print(f"投票集成准确率: {voting_acc:.4f}")

投票集成准确率: 1.0000


在这段代码中，我们创建了一个投票分类器 (`VotingClassifier`) 来集成多个基础分类器的预测结果。以下是代码的详细解释：

1. **`VotingClassifier`**:
   - `VotingClassifier` 是 Scikit-learn 提供的一种集成学习方法，用于结合多个分类器的预测结果以提高整体模型的性能。
   - 它支持两种投票方式：硬投票 (`hard voting`) 和软投票 (`soft voting`)。
     - 硬投票：根据每个分类器的预测结果进行多数表决，选择票数最多的类别。
     - 软投票：根据每个分类器的预测概率进行加权平均，选择概率最高的类别。

2. **`estimators` 参数**:
   - 这是一个包含多个基础分类器的列表，每个分类器由一个名称和一个模型实例组成。
   - 在这里，我们定义了三个基础分类器：
     - `'lin_reg'`: 一个逻辑回归分类器 (`LogisticRegression`)，最大迭代次数设置为 1000。
     - `'knn'`: 一个 K 最近邻分类器 (`KNeighborsClassifier`)，邻居数设置为 5。
     - `'log_reg'`: 另一个逻辑回归分类器，使用 L2 正则化，最大迭代次数设置为 1000。

3. **`voting` 参数**:
   - 这里设置为 `'hard'`，表示使用硬投票方式。每个基础分类器对样本进行预测后，最终的预测结果由多数表决决定。

通过这种方式，投票分类器可以结合多个模型的优势，从而提高分类的准确性和鲁棒性。

随机森林

In [446]:


# 随机森林模型
rf_clf = RandomForestClassifier(
    n_estimators=100,  # 树的数量
    max_depth=None,    # 树的最大深度
     random_state=42
)
rf_clf.fit(x_train, y_train)  # 随机森林不需要特征缩放

# 评估
rf_acc = rf_clf.score(X_test, y_test)
print(f"随机森林准确率: {rf_acc:.4f}")

随机森林准确率: 0.3667


在这段代码中，我们创建了一个随机森林分类器 (`RandomForestClassifier`) 来进行分类任务。以下是代码的详细解释：

1. **`RandomForestClassifier`**:
   - 随机森林是一种集成学习方法，通过构建多个决策树并结合它们的预测结果来提高模型的准确性和鲁棒性。
   - 它通过引入随机性（如随机选择特征和样本）来减少过拟合问题。

2. **参数说明**:
   - `n_estimators=100`:
     - 指定随机森林中包含的决策树数量。在这里，我们设置为 100 棵树。
     - 更多的树通常可以提高模型的性能，但也会增加计算成本。
   - `max_depth=None`:
     - 指定每棵决策树的最大深度。`None` 表示树会一直生长，直到所有叶子节点是纯的，或者叶子节点包含的样本数小于 `min_samples_split`。
     - 限制树的深度可以防止过拟合。
   - `random_state=42`:
     - 设置随机种子，以确保结果的可重复性。

3. **随机森林的特点**:
   - 随机森林通过对数据集进行多次采样（袋外采样，Bootstrap Sampling）来训练每棵树。
   - 每棵树在训练时只使用部分特征，从而增加了模型的多样性。
   - 最终的预测结果是通过所有树的投票（分类任务）或平均值（回归任务）得出的。

通过这种方式，随机森林能够在保持高准确率的同时，减少单一决策树可能出现的过拟合问题，是一种强大的集成学习方法。

Adaboost

In [447]:


# AdaBoost模型
ada_clf = AdaBoostClassifier(
    n_estimators=50,    # 弱学习器的最大数量
    learning_rate=1.0,  # 学习率
    random_state=42
)
ada_clf.fit(X_train, y_train)  # AdaBoost也不需要特征缩放

# 评估
ada_acc = ada_clf.score(x_test, y_test)
print(f"AdaBoost准确率: {ada_acc:.4f}")

AdaBoost准确率: 0.2000


在这段代码中，我们创建了一个 AdaBoost 分类器 (`AdaBoostClassifier`) 来进行分类任务。以下是代码的详细解释：

1. **`AdaBoostClassifier`**:
   - AdaBoost（Adaptive Boosting）是一种提升方法，通过组合多个弱学习器（通常是决策树）来构建一个强大的分类器。
   - 它通过迭代地调整样本权重，使模型更加关注之前分类错误的样本，从而提高整体性能。

2. **参数说明**:
   - `n_estimators=50`:
     - 指定弱学习器的最大数量。在这里，我们设置为 50 个弱学习器。
     - 更多的弱学习器可能会提高模型的性能，但也可能导致过拟合。
   - `learning_rate=1.0`:
     - 学习率控制每个弱学习器对最终模型的贡献。较小的学习率可以提高模型的鲁棒性，但需要更多的弱学习器。
   - `random_state=42`:
     - 设置随机种子，以确保结果的可重复性。

3. **AdaBoost 的特点**:
   - 每次迭代中，AdaBoost 会根据前一轮的分类结果调整样本的权重，增加分类错误样本的权重。
   - 最终的分类结果是通过加权投票（分类任务）或加权平均（回归任务）得出的。

通过这种方式，AdaBoost 能够有效地提升弱学习器的性能，适用于处理噪声较少的数据集。

In [448]:
import pandas as pd

results = pd.DataFrame({
    'Model': ['支持向量机', 'KNN', '逻辑回归', '投票集成', '随机森林', 'AdaBoost'],
    'Accuracy': [svm_acc, knn_acc, log_acc, voting_acc, rf_acc, ada_acc]
})

print(results.sort_values(by='Accuracy', ascending=False))

      Model  Accuracy
0     支持向量机  1.000000
3      投票集成  1.000000
1       KNN  0.966667
2      逻辑回归  0.966667
4      随机森林  0.366667
5  AdaBoost  0.200000
